# Akka.Streams and Backoff
Part of the raison d'être for Akka.Streams in the first place is to faciliate support for [Reactive Streams](https://www.reactive-streams.org/) - the ability to ensure that in asynchronous producer-consumer systems a faster upstream producer can't overwhelm a slower downstream consumer.

Let's consider an example.

In [ ]:
#r "nuget: Akka.Streams, 1.4.27"
#r "nuget: Akka.Streams.IAsyncEnumerable, 0.1.0"

using System.Linq;
using System.Collections.Immutable;
using Akka;
using Akka.Actor;
using Akka.Streams;
using Akka.Streams.Dsl;

ActorSystem actorSystem = ActorSystem.Create("StreamsExample2");
IMaterializer materializer = actorSystem.Materializer();

// Fast upstream producer - instantly produces 10,000 requests
Source<int,NotUsed> source = Source.From(Enumerable.Range(0, 10000));

// Group into batches of 100
Source<IEnumerable<int>,NotUsed> batchingSource = source.Via(Flow.Create<int>().Grouped(100)); 

var start = DateTime.UtcNow;

// simulate a slower downstream consumer - can only process 10 events per second
var slowSink = batchingSource.Via(Flow.Create<IEnumerable<int>>().Delay(TimeSpan.FromMilliseconds(1000), DelayOverflowStrategy.Backpressure)
    .Select(x => (x.Sum(), DateTime.UtcNow - start))); // sum each group;

var output = slowSink.RunAsAsyncEnumerable(materializer);

await foreach(var i in output){
    Console.WriteLine($"{i}");
}


Installed Packages Akka.Streams, 1.4.27 Akka.Streams.IAsyncEnumerable, 0.1.0

(4950, 00:00:01.0256786)
(14950, 00:00:01.0260298)
(24950, 00:00:01.0260323)
(34950, 00:00:01.0260343)
(44950, 00:00:01.0260409)
(54950, 00:00:01.0260426)
(64950, 00:00:01.0260444)
(74950, 00:00:01.0260461)
(84950, 00:00:01.0260478)
(94950, 00:00:01.0260495)
(104950, 00:00:01.0260514)
(114950, 00:00:01.0260531)
(124950, 00:00:01.0260548)
(134950, 00:00:01.0260566)
(144950, 00:00:01.0260583)
(154950, 00:00:01.0260602)
(164950, 00:00:02.0371888)
(174950, 00:00:02.0375004)
(184950, 00:00:02.0375037)
(194950, 00:00:02.0375063)
(204950, 00:00:02.0375145)
(214950, 00:00:02.0375166)
(224950, 00:00:02.0375186)
(234950, 00:00:02.0375205)
(244950, 00:00:02.0375225)
(254950, 00:00:02.0375245)
(264950, 00:00:02.0375265)
(274950, 00:00:02.0375285)
(284950, 00:00:02.0375305)
(294950, 00:00:02.0375323)
(304950, 00:00:02.0375343)
(314950, 00:00:02.0375363)
(324950, 00:00:03.0552662)
(334950, 00:00:03.0555440)
(344950, 00:00:03.0555481)
(354950, 00:00:03.0555507)
(364950, 00:00:03.0555531)
(374950, 00:

In this instance, we are going to simulate a slower downstream consumer by using [a `Delay<T>` flow](https://getakka.net/api/Akka.Streams.Dsl.DelayFlow-1.html), which creates 100ms pauses in the stream:

```csharp
Flow.Create<IEnumerable<int>>().Delay(TimeSpan.FromMilliseconds(100))
```

The upstream producer, our `Source<int>`, on the other hand, will immediately create 10,000 items that can be processed immediately:

```csharp
Source<int,NotUsed> source = Source.From(Enumerable.Range(0, 10000));
```

So what do we do? In our case, we need to equip our Akka.Streams graph with the ability to handle backpressure somehow - otherwise, this stream will take 1,000 seconds to run through to completion.

The approach we used in the sample above to do this is [a `Grouped<T,IEnumerable<T>>` flow](https://getakka.net/api/Akka.Streams.Dsl.FlowOperations.html#Akka_Streams_Dsl_FlowOperations_Grouped__3_Akka_Streams_Dsl_Flow___0___1___2__System_Int32_) - which will chunk upstream elements into groups of up to 100. When the `Delay<T>` backpressures, however, the grouping will stop and the original `Source<int>` will stop being read from until the downstream is ready to process again.

That's why you can see this type of pattern appear in the output:

```
Sum,   DateTime.UtcNow.Ticks
(4950, 637654561536058261)
(14950, 637654561536077747)
(24950, 637654561536078125)
```

The sums are growing larger because we're selecting larger integers in the sequence - and the tickcount is growing larger because the processing delay in computing the sum is being observed in real-time as the stream is processed.

This is an example of backpressure at work inside an Akka.NET Stream!